# Learning Dynamics and Lyapunov function for Circle Path Following (Unicycle)

In [ ]:
# To install dReal, if you use Google CoLab
# If not CoLab, command out

import pkgutil
if not pkgutil.find_loader("dreal"):
  !curl https://raw.githubusercontent.com/dreal/dreal4/master/setup/ubuntu/18.04/install.sh | bash
  !pip install dreal --upgrade

In [ ]:
# -*- coding: utf-8 -*-
from dreal import *
from Functions import *
import torch 
import torch.nn.functional as F
import numpy as np
import timeit 
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Actual dynamical system

In [ ]:
def f_value(x,u):
    v = 1
    y = torch.zeros_like(x)
    y[:,0] = v*torch.sin(x[:,1])
    y[:,1] = u[:,0] - v*torch.cos(x[:,1])/(1-x[:,0])
    return y

## NN for the dynamics

In [ ]:
# NN for dynamics  1 hidden

class fNet(torch.nn.Module):
    def __init__(self,n_input, n_hidden1,  n_output):
        super().__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden1)
        self.layer2 = torch.nn.Linear(n_hidden1,n_output)  
        self.to(device)  

    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = self.layer2(h_1) 
        return out

## Learning the dynamics with NNs

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
# generate training dataset
N_f = 1801
xx = np.linspace(-0.9,0.9,N_f, dtype = float)
x_f = []
for i in range(0,N_f): 
    for j in range(0,N_f):
        x_f.append([xx[j],xx[i]])

x_f = torch.tensor(x_f)
x_f = x_f.float()
ut_bdd = 5.625  # bound for input
u_t = torch.Tensor(len(x_f), 1).uniform_(-ut_bdd, ut_bdd) 

# target
t_f = f_value(x_f,u_t)

# input of FNN
x_train = torch.cat((x_f,u_t),1)

In [ ]:
# define parameters
max_iter = 1000
losses = []
# NN: 1 hidden layers with 200 neurons 
fnet = fNet(n_input = 3, n_hidden1 = 200,  n_output = 2) 
optimizer = torch.optim.Adam(fnet.parameters(), lr = 0.05)

loss_func = torch.nn.MSELoss(reduction='sum')

# # training
for epoch in tqdm(range(max_iter)):
    x_train = x_train.to(device)
    t_f = t_f.to(device)
    y_nn = fnet(x_train)
    loss = loss_func(y_nn,t_f)
    losses.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    with torch.no_grad():
        torch.cuda.empty_cache()

plt.plot(losses)

In [ ]:
losses[-1]

In [ ]:
# train more epoches
# fnet = fnet.to(device) switch between devices if needed
losses = []
optimizer = torch.optim.Adam(fnet.parameters(), lr = 0.001)
loss_func = torch.nn.MSELoss(reduction='sum')

for epoch in tqdm(range(1000)):
    x_train = x_train.to(device)
    t_f = t_f.to(device)
    y_nn = fnet(x_train)
    loss = loss_func(y_nn,t_f)
    losses.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        torch.cuda.empty_cache()

plt.plot(losses)

In [ ]:
plt.plot(losses[-500:-1])

In [ ]:
# generate testing dataset
r = 0.8 # region of interest
N_l = 1601 # change according to delta
xl = np.linspace(-r,r,N_l, dtype = float)
u_bdd = 5.  # bound for input

In [ ]:
# check the rough value of alpha
x_l = []
N_l = 1601 
for i in range(0,N_l): 
    for j in range(0,N_l):
        x_l.append([xl[j],xl[i]])

x_l = torch.tensor(x_l)
x_l = x_l.float()

u_l = torch.Tensor(len(x_l), 1).uniform_(-u_bdd, u_bdd) 

# target
t_l = f_value(x_l,u_l)
x_bdd = torch.cat((x_l,u_l),1)

# output of FNN
x_bdd = x_bdd.to(device)
t_l = t_l.to(device)
# fnet = fnet.to(device) # if need to switch device
y_l = fnet(x_bdd)

# maximum of loss
loss_all = torch.norm(y_l-t_l, dim = 1)
alpha = torch.max(loss_all)

# # infnity norm if needed
# dist = y_l-t_l
# loss_all = torch.linalg.norm(dist, float('inf'))
# torch.max(loss_all)

In [ ]:
# check the actual value of alpha 
# Remark (!): this box takes hours/days to run, please use the above line if you just need a rough value of alpha 

N_u = 50001  # 100001 corresponds to 1e-4
u_l = np.linspace(-u_bdd,u_bdd, N_u, dtype = float).reshape(N_u,1)
u_l = torch.tensor(u_l)
alpha = torch.zeros(N_l,N_l)

for i in tqdm(range(0,N_l)): 
    for j in range(0,N_l):
        x1 = xl[j]*torch.ones_like(u_l)
        x2 = xl[i]*torch.ones_like(u_l)
        x_l = torch.cat((x1,x2), dim = 1)
        t_l = f_value(x_l,u_l)
        X_l = torch.cat((x_l,u_l),1)
        X_l = X_l.float()
        X_l = X_l.to(device)
        t_l = t_l.to(device)
        with torch.no_grad():
            y_l = fnet(X_l)

        torch.cuda.empty_cache()
        # target
        loss_all = torch.norm(y_l-t_l, dim = 1)
        del y_l
        max = torch.max(loss_all)
        alpha[i,j] = max

alpha_max = torch.max(alpha)


In [ ]:
# save the weights to calculate the Lipschitz constant with LipSDP
f_w1 = fnet.layer1.weight.data.cpu().numpy()
f_w2 = fnet.layer2.weight.data.cpu().numpy()

np.savetxt("fw1.txt", f_w1, fmt="%s")
np.savetxt("fw2.txt", f_w2, fmt="%s")

In [ ]:
# save the fNN
# torch.save(fnet.cpu(), 'PF_fnet.pt')

# load fNN from a file
# fnet = torch.load('PF_fnet.pt').to(device)

## Plot the approximated results

In [ ]:
# dataset for plot
N_p = 300
xp = np.linspace(-0.8,0.8,N_p, dtype = float).reshape(N_p,1)
xp = np.concatenate((xp,xp), axis=1 ) 
xp = torch.tensor(xp)
xp = xp.float()
u_0 = np.zeros(len(xp)).reshape(len(xp),1)
u_0 = torch.tensor(u_0)

# target of plotting
tp = f_value(xp,u_0)

# input of plotting
Xp_0 = torch.cat((xp,u_0), 1)    
Xp_0 = Xp_0.float()


In [ ]:
# output of plotting
# fnet_1 = fnet.to('cpu') # to plot, command out this line
yp = fnet_1(Xp_0)

with torch.no_grad():
    plt.figure(1)
    plt.plot(xp[:,0],yp[:,0],label='NN')
    plt.plot(xp[:,0],tp[:,0], label='y_actual')
    plt.legend(loc = 0)
    plt.xlabel('x1&x2 pair')
    plt.ylabel('f1')

    plt.figure(2)
    plt.plot(xp[:,0],yp[:,1],label='NN')
    plt.plot(xp[:,0],tp[:,1],label='y_actual')
    plt.legend()
    plt.xlabel('x1&x2 pair')
    plt.ylabel('f2')

## Learned dynamics

In [ ]:
def f_learned(x,u):
    X = torch.cat((x,u),1)
    y = fnet(X)
    return y

## Neural network model
Building NN with random parameters for Lyapunov function and initializing parameters of NN controller to LQR solution

LQR solution is obtained by minimizing the cost function J = ∫(xᵀQx + uᵀRu)dt, where Q is 2×2 identity matrix and R is 1×1 identity matrix

In [ ]:
class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output,lqr,b):
        super(Net, self).__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden)
        self.layer2 = torch.nn.Linear(n_hidden,n_output)
        self.control = torch.nn.Linear(n_input,1)
        self.control.weight = torch.nn.Parameter(lqr)
        self.control.bias = torch.nn.Parameter(b)
        self.control.bias.requires_grad = False
        self.to(device)
        
    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = sigmoid(self.layer2(h_1))
        u = 5*sigmoid(self.control(x))
        return out,u

## Parameters

In [ ]:
'''
For learning 
'''
N = 500            # sample size
D_in = 2            # input dimension
H1 = 6              # hidden dimension
D_out = 1           # output dimension
torch.manual_seed(10)  

r = 0.8
lqr = torch.tensor([[-2.4142, -2.4142]])  # lqr solution
b = torch.tensor([0.1974])
x = torch.Tensor(N, D_in).uniform_(-r, r)           
x_0 = torch.zeros([1, 2])
x_0 = x_0.to(device)

'''
For verifying 
'''
x1 = Variable("x1")
x2 = Variable("x2")
vars_ = [x1,x2]
v = 1
l = 1
config = Config()
config.use_polytope_in_forall = True
config.use_local_optimization = True
config.precision = 1e-2
beta = -0.1
# Checking candidate V within a ball around the origin (ball_lb ≤ sqrt(∑xᵢ²) ≤ ball_ub)
ball_lb = 0.1
ball_ub = r

# parameters for beta
Kf = 45.
KF = 108.
d = 1e-4
loss = 0.007 # equals alpha above

## Learning and Falsification

In [ ]:
out_iters = 0
valid = False
while out_iters < 2 and not valid: 
    start = timeit.default_timer()
    lqr = torch.tensor([[-2.4142, -2.4142]])   # lqr solution
    b = torch.tensor([0.1974])
    model = Net(D_in,H1, D_out,lqr,b)
    L = []
    i = 0 
    t = 0
    max_iters = 3000 # increase number of epoches if cannot find a valid LF
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # learned dynamics
    f_w1 = fnet.layer1.weight.data.cpu().numpy()
    f_w2 = fnet.layer2.weight.data.cpu().numpy()
    f_b1 = fnet.layer1.bias.data.cpu().numpy()
    f_b2 = fnet.layer2.bias.data.cpu().numpy()

    while i < max_iters and not valid: 
        x = x.float()
        x = x.to(device)
        V_candidate, u = model(x)
        X0,u0 = model(x_0)
        f = f_learned(x,u)
        Circle_Tuning = Tune(x)
        Circle_Tuning = Circle_Tuning.to(device)
        # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
        L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0)

        dVdx = torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight)

        # With tuning

        Lyapunov_risk = (F.relu(-V_candidate)+ 1.2*F.relu(L_V+0.2)).mean()\
                 +0.5*((Circle_Tuning-V_candidate).pow(2)).mean()+ 1.2*(X0).pow(2) + 0.0001*torch.norm(dVdx)


        print(i, "Lyapunov Risk=",Lyapunov_risk.item()) 
        L.append(Lyapunov_risk.item())
        optimizer.zero_grad()
        Lyapunov_risk.backward()
        optimizer.step() 

        # neural Lyapunov function
        w1 = model.layer1.weight.data.cpu().numpy()
        w2 = model.layer2.weight.data.cpu().numpy()
        b1 = model.layer1.bias.data.cpu().numpy()
        b2 = model.layer2.bias.data.cpu().numpy()
        q = model.control.weight.data.cpu().numpy()
        b = model.control.bias.data.cpu().numpy()

        # Falsification
        if i % 10 == 0:
            u_NN = 5*tanh(q.item(0)*x1 + q.item(1)*x2 + b)
            print(u_NN)
            vars_nn = [x1, x2, 5*tanh(q.item(0)*x1 + q.item(1)*x2 + b) ]

            f_h1 = []
            
            f_z1 = np.dot(vars_nn,f_w1.T)+f_b1
            for n in range(len(f_z1)):
                f_h1.append(tanh(f_z1[n]))

            f_learn = np.dot(f_h1,f_w2.T)+f_b2

            # Candidate V
            z1 = np.dot(vars_,w1.T)+b1

            a1 = []
            for j in range(0,len(z1)):
                a1.append(tanh(z1[j]))
            z2 = np.dot(a1,w2.T)+b2
            V_learn = tanh(z2.item(0))

            print('===========Verifying==========')        
            start_ = timeit.default_timer() 
            beta = np.maximum(beta, -0.1) # in case beta is too negative and cannot return any results
            result= CheckLyapunov(vars_, f_learn, V_learn, ball_lb, ball_ub, config, beta)
            stop_ = timeit.default_timer() 

            if (result): 
                print("Not a Lyapunov function. Found counterexample: ")
                print(result)
                x = x.to('cpu')
                x = AddCounterexamples(x,result,10)
            else:  
                # calculate norm of dVdx with the SMT solver
                M = 0.5 # lower bound of M
                violation = CheckdVdx(vars_, V_learn, ball_ub, config, M) 
                while violation:
                    violation = CheckdVdx(vars_, V_learn, ball_ub, config, M)
                    if not violation:
                        dvdx_bound = np.sqrt(M)
                        print(dvdx_bound, "is the norm of dVdx")
                    M += 0.01
                beta = -dvdx_bound*((Kf+KF)*d+loss) # update beta 
                result_strict= CheckLyapunov(vars_, f_learn, V_learn, ball_lb, ball_ub, config, beta) # SMT solver
                if not result_strict:
                    valid = True
                    print("Satisfy conditions with beta = ", beta)
                    print(V_learn, " is a Lyapunov function.")
            t += (stop_ - start_)
            print('==============================') 
        i += 1

    stop = timeit.default_timer()

    np.savetxt("pf_w1.txt", model.layer1.weight.data.cpu(), fmt="%s")
    np.savetxt("pf_w2.txt", model.layer2.weight.data.cpu(), fmt="%s")
    np.savetxt("pf_b1.txt", model.layer1.bias.data.cpu(), fmt="%s")
    np.savetxt("pf_b2.txt", model.layer2.bias.data.cpu(), fmt="%s")
    np.savetxt("pf_K.txt", model.control.weight.data.cpu(), fmt="%s")

    print('\n')
    print("Total time: ", stop - start)
    print("Verified time: ", t)
    
    out_iters+=1

### Checking result with a certain beta

In [ ]:
epsilon = -0.1
start_ = timeit.default_timer() 
result = CheckLyapunov(vars_, f_learn, V_learn, ball_lb, ball_ub, config, beta)
stop_ = timeit.default_timer() 

if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
    print(result)
else:  
    print("Satisfy conditions with beta = ",beta)
    print(V_learn, " is a Lyapunov function.")
t += (stop_ - start_)

## Checking with epsilon = 0 for actual dynamics

In [ ]:
beta = 0
f_u = [v*sin(x2),
     u_NN -v*(cos(x2)/(1-x1))]
start_ = timeit.default_timer() 
result = CheckLyapunov(vars_, f_u, V_learn, ball_lb, ball_ub, config, beta)
stop_ = timeit.default_timer() 
if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
    print(result)
else:  
    print("Satisfy conditions with beta = ",beta)
    print(V_learn, " is a Lyapunov function for actual dynamics.")
t += (stop_ - start_)
